# Heuristics

This notebook contains code examples referring to the Heuristics chapter of "Applied Mathematics with Open-Source Software: Operational Research Problems
with Python and R".

First we write a function to generate an initial random candidate solution.

In [1]:
import numpy as np


def get_initial_candidate(number_of_stops, seed):
    """Return an random initial tour.

    Args:
        number_of_stops: The number of stops
        seed: An integer seed.

    Returns:
        A tour starting an ending at stop with index 0.
    """
    internal_stops = list(range(1, number_of_stops))
    np.random.seed(seed)
    np.random.shuffle(internal_stops)
    return [0] + internal_stops + [0]

An example with 13 stops.

In [2]:
number_of_stops = 13
seed = 0
initial_candidate = get_initial_candidate(
    number_of_stops=number_of_stops,
    seed=seed,
)
print(initial_candidate)

[0, 7, 12, 5, 11, 3, 9, 2, 8, 10, 4, 1, 6, 0]


Next we write a function that calculates the cost of that solution.

In [3]:
def get_cost(tour, distance_matrix):
    """Return the cost of a tour.

    Args:
        tour: A given tuple of successive stops.
        distance_matrix: The distance matrix of the problem.

    Returns:
        The cost
    """
    return sum(
        distance_matrix[current_stop, next_stop]
        for current_stop, next_stop in zip(tour[:-1], tour[1:])
    )

For the particular distance matrix given in the chapter, the cost of the initial solution is:

In [4]:
distance_matrix = np.array(
    (
        (0, 35, 35, 29, 70, 35, 42, 27, 24, 44, 58, 71, 69),
        (35, 0, 67, 32, 72, 40, 71, 56, 36, 11, 66, 70, 37),
        (35, 67, 0, 63, 64, 68, 11, 12, 56, 77, 48, 67, 94),
        (29, 32, 63, 0, 93, 8, 71, 56, 8, 33, 84, 93, 69),
        (70, 72, 64, 93, 0, 101, 56, 56, 92, 81, 16, 5, 69),
        (35, 40, 68, 8, 101, 0, 76, 62, 11, 39, 91, 101, 76),
        (42, 71, 11, 71, 56, 76, 0, 15, 65, 81, 40, 60, 94),
        (27, 56, 12, 56, 56, 62, 15, 0, 50, 66, 41, 58, 82),
        (24, 36, 56, 8, 92, 11, 65, 50, 0, 39, 81, 91, 74),
        (44, 11, 77, 33, 81, 39, 81, 66, 39, 0, 77, 79, 37),
        (58, 66, 48, 84, 16, 91, 40, 41, 81, 77, 0, 20, 73),
        (71, 70, 67, 93, 5, 101, 60, 58, 91, 79, 20, 0, 65),
        (69, 37, 94, 69, 69, 76, 94, 82, 74, 37, 73, 65, 0),
    )
)
cost = get_cost(
    tour=initial_candidate,
    distance_matrix=distance_matrix,
)
print(cost)

827


We then write a function to act as a neighbourhood operator. This one swaps two random stops on the tour.

In [5]:
def swap_stops(tour):
    """Return a new tour by swapping two stops.

    Args:
        tour: A given tuple of successive stops.

    Returns:
        A tour
    """
    number_of_stops = len(tour) - 1
    i, j = np.random.choice(range(1, number_of_stops), 2)
    new_tour = list(tour)
    new_tour[i], new_tour[j] = tour[j], tour[i]
    return new_tour

Applying this to the initial solution:

In [6]:
print(swap_stops(initial_candidate))

[0, 7, 12, 5, 11, 3, 9, 2, 8, 1, 4, 10, 6, 0]


Now we combine these and define a function that runs the neighbourhood search algorithm.

In [7]:
def run_neighbourhood_search(
    distance_matrix,
    iterations,
    seed,
    neighbourhood_operator=swap_stops,
):
    """Returns a tour by carrying out a neighbourhood search.

    Args:
        distance_matrix: the distance matrix
        iterations: the number of iterations for which to
                    run the algorithm
        seed: a random seed
        neighbourhood_operator: the neighbourhood operator
                                (default: swap_stops)

    Returns:
        A tour
    """
    number_of_stops = len(distance_matrix)
    candidate = get_initial_candidate(
        number_of_stops=number_of_stops,
        seed=seed,
    )
    best_cost = get_cost(
        tour=candidate,
        distance_matrix=distance_matrix,
    )
    for _ in range(iterations):
        new_candidate = neighbourhood_operator(candidate)
        cost = get_cost(
            tour=new_candidate,
            distance_matrix=distance_matrix,
        )
        if cost <= best_cost:
            best_cost = cost
            candidate = new_candidate

    return candidate

Running this for 1000 iterations gives a better solution.

In [8]:
number_of_iterations = 1000

solution_with_swap_stops = run_neighbourhood_search(
    distance_matrix=distance_matrix,
    iterations=number_of_iterations,
    seed=seed,
    neighbourhood_operator=swap_stops,
)
print(solution_with_swap_stops)

[0, 7, 2, 8, 5, 3, 1, 9, 12, 11, 4, 10, 6, 0]


with cost:

In [9]:
cost = get_cost(
    tour=solution_with_swap_stops,
    distance_matrix=distance_matrix,
)
print(cost)

362


We now define a different neighbourhood operator, reversing the tour between two stops.

In [10]:
def reverse_path(tour):
    """Return a new tour by reversing the path between two stops.

    Args:
        tour: A given tuple of successive stops.

    Returns:
        A tour
    """
    number_of_stops = len(tour) - 1
    stops = np.random.choice(range(1, number_of_stops), 2)
    i, j = sorted(stops)
    new_tour = tour[:i] + tour[i : j + 1][::-1] + tour[j + 1 :]
    return new_tour

Applying this to the initial solution:

In [11]:
print(reverse_path(initial_candidate))

[0, 7, 4, 10, 8, 2, 9, 3, 11, 5, 12, 1, 6, 0]


Using this neighbourhood operator in the neighbourhood search instead, gives an even better solution.

In [12]:
solution_with_reverse_path = run_neighbourhood_search(
    distance_matrix=distance_matrix,
    iterations=number_of_iterations,
    seed=seed,
    neighbourhood_operator=reverse_path,
)
print(solution_with_reverse_path)

[0, 8, 5, 3, 1, 9, 12, 11, 4, 10, 6, 2, 7, 0]


with cost:

In [13]:
cost = get_cost(
    tour=solution_with_reverse_path,
    distance_matrix=distance_matrix,
)
print(cost)

299
